In [ ]:
#!/usr/bin/env python3
"""
Turn-based battle system (terminal).
Save as battle_game.py and run: python battle_game.py

Features:
- Player and enemy Characters with stats (HP, Attack, Defense, Speed)
- Moves with damage, healing, buffs/debuffs, cooldowns
- Status effects: burn (damage over time), stun (skip turn)
- Items (healing potion, antidote)
- Simple enemy AI
- XP & leveling system
- Input validation and help
"""

import random
import time
import os
from dataclasses import dataclass, field
from typing import List, Optional, Callable, Dict

# ---- Utilities ----
def clear_screen():
    os.system('cls' if os.name == 'nt' else 'clear')

def clamp(v, a, b):
    return max(a, min(b, v))

# ---- Game Data Classes ----
@dataclass
class Status:
    name: str
    duration: int
    source: Optional[str] = None
    potency: int = 0  # used for e.g. burn damage or buff magnitude

@dataclass
class Move:
    name: str
    desc: str
    power: int = 0               # base damage (can be negative for heal)
    accuracy: float = 1.0        # 0.0 - 1.0
    cooldown: int = 0            # turns between use
    current_cd: int = 0          # remaining cooldown
    target_self: bool = False
    apply_status: Optional[Callable[['Character','Character'], Optional[Status]]] = None
    buff_stat: Optional[str] = None
    buff_amount: int = 0
    buff_duration: int = 0

    def ready(self):
        return self.current_cd <= 0

    def trigger_cd(self):
        self.current_cd = self.cooldown

    def tick_cd(self):
        if self.current_cd > 0:
            self.current_cd -= 1

@dataclass
class Item:
    name: str
    desc: str
    use: Callable[['Character'], str]
    qty: int = 1

@dataclass
class Character:
    name: str
    level: int
    max_hp: int
    hp: int
    atk: int
    defense: int
    speed: int
    moves: List[Move]
    statuses: List[Status] = field(default_factory=list)
    buffs: Dict[str, Status] = field(default_factory=dict)  # keyed by stat name
    xp: int = 0

    def is_alive(self):
        return self.hp > 0

    def take_damage(self, amount: int):
        amt = max(0, amount)
        self.hp = clamp(self.hp - amt, 0, self.max_hp)
        return amt

    def heal(self, amount: int):
        self.hp = clamp(self.hp + amount, 0, self.max_hp)
        return amount

    def add_status(self, status: Status):
        # If same status exists, refresh duration
        for s in self.statuses:
            if s.name == status.name:
                s.duration = status.duration
                s.potency = status.potency
                return
        self.statuses.append(status)

    def add_buff(self, stat: str, amount: int, duration: int):
        self.buffs[stat] = Status(name=f"buff:{stat}", duration=duration, potency=amount)

    def get_stat(self, stat_name: str):
        base = getattr(self, stat_name)
        buff = self.buffs.get(stat_name)
        if buff:
            return max(0, base + buff.potency)
        return base

    def tick_statuses(self):
        """Apply status effects at start of turn; return messages list."""
        msgs = []
        # apply effects like burn
        for s in self.statuses[:]:
            if s.name == 'Burn':
                dmg = s.potency
                self.take_damage(dmg)
                msgs.append(f"{self.name} takes {dmg} burn damage! ({self.hp}/{self.max_hp} HP)")
            # other statuses could be handled here
            s.duration -= 1
            if s.duration <= 0:
                msgs.append(f"{self.name} is no longer {s.name.lower()}.")
                self.statuses.remove(s)
        # tick buff durations
        for k, b in list(self.buffs.items()):
            b.duration -= 1
            if b.duration <= 0:
                msgs.append(f"{self.name}'s {k} buff wore off.")
                del self.buffs[k]
        return msgs

    def has_status(self, status_name: str):
        return any(s.name == status_name for s in self.statuses)

    def tick_move_cds(self):
        for m in self.moves:
            m.tick_cd()

# ---- Moves and Items Setup ----
def make_player_moves():
    return [
        Move(
            name="Slash",
            desc="A solid physical strike.",
            power=18, accuracy=0.95, cooldown=0
        ),
        Move(
            name="Power Strike",
            desc="Heavy hit with lower accuracy.",
            power=30, accuracy=0.75, cooldown=2
        ),
        Move(
            name="First Aid",
            desc="Heal yourself for moderate HP.",
            power=-25, accuracy=1.0, cooldown=3, target_self=True
        ),
        Move(
            name="Flame Shot",
            desc="Deal fire damage and may burn.",
            power=15, accuracy=0.9, cooldown=3,
            apply_status=lambda user, target: Status(name="Burn", duration=3, potency=5)
        )
    ]

def make_enemy_moves(level):
    # Enemies can have scaled moves
    return [
        Move(name="Bite", desc="Enemy bite.", power=12 + level*2, accuracy=0.95),
        Move(name="Rage", desc="Powerful blow when angry.", power=20 + level*3, accuracy=0.8, cooldown=2),
        Move(name="Poison Spit", desc="May inflict burn-like poison (we'll use Burn)", power=8 + level, accuracy=0.85,
             apply_status=lambda u, t: Status(name="Burn", duration=3, potency=4))
    ]

def make_items():
    return {
        'potion': Item(name='Potion', desc='Heals 40 HP.', qty=3,
                       use=lambda c: f"{c.name} healed {c.heal(40)} HP."),
        'antidote': Item(name='Antidote', desc='Cures Burn.', qty=1,
                         use=lambda c: _use_antidote(c))
    }

def _use_antidote(c: Character):
    removed = False
    for s in c.statuses[:]:
        if s.name == 'Burn':
            c.statuses.remove(s)
            removed = True
    return f"{c.name} had their burn removed." if removed else f"{c.name} had no status to remove."

# ---- Combat Resolution ----
def calculate_damage(attacker: Character, defender: Character, base_power: int):
    # Basic formula: (Attack * power / Defense) * randomness
    atk = attacker.get_stat('atk')
    defense = defender.get_stat('defense')
    if defense <= 0:
        defense = 1
    raw = (atk * base_power) / defense
    variation = random.uniform(0.85, 1.15)
    dmg = int(max(1, raw * variation))
    return dmg

def attempt_hit(move: Move):
    return random.random() <= move.accuracy

# Simple enemy AI chooses an available move (prefers damaging moves but uses heal if low hp)
def enemy_choose_move(enemy: Character, player: Character):
    usable = [m for m in enemy.moves if m.ready()]
    if not usable:
        return None
    # If low HP and has low-power move that heals? (enemy moves here don't heal, but this logic is generic)
    if enemy.hp < enemy.max_hp * 0.35:
        # pick lower-power moves or special; fallback to random
        candidates = sorted(usable, key=lambda mm: mm.power)
        return candidates[0]
    # prefer high power
    candidates = sorted(usable, key=lambda mm: mm.power, reverse=True)
    # small chance to choose a status move
    if random.random() < 0.2:
        for m in usable:
            if m.apply_status:
                return m
    return candidates[0]

# ---- Battle Loop ----
def battle(player: Character, enemy: Character, inventory: Dict[str, Item]):
    clear_screen()
    print(f"--- A wild {enemy.name} (Lv {enemy.level}) appears! ---\n")
    time.sleep(0.6)

    while player.is_alive() and enemy.is_alive():
        # Start-of-turn status ticks
        for msg in player.tick_statuses():
            print(msg)
            time.sleep(0.3)
        for msg in enemy.tick_statuses():
            print(msg)
            time.sleep(0.3)

        # Check stun or similar
        player_stunned = player.has_status('Stun')
        enemy_stunned = enemy.has_status('Stun')

        # Show HUD
        print_hud(player, enemy, inventory)

        # Decide turn order by speed
        actors = sorted([('player', player), ('enemy', enemy)], key=lambda x: x[1].get_stat('speed'), reverse=True)

        for tag, actor in actors:
            if not player.is_alive() or not enemy.is_alive():
                break  # someone died during this round

            if tag == 'player':
                if player_stunned:
                    print(f"{player.name} is stunned and skips a turn!")
                    player_stunned = False  # stun expires as tick reduced duration earlier
                    time.sleep(0.7)
                else:
                    player_action(player, enemy, inventory)
            else:
                if enemy_stunned:
                    print(f"{enemy.name} is stunned and skips a turn!")
                    enemy_stunned = False
                    time.sleep(0.7)
                else:
                    enemy_action(enemy, player)

            # tick cooldowns after each action
            player.tick_move_cds()
            enemy.tick_move_cds()

        # small delay between rounds
        print("\n--- Next round ---\n")
        time.sleep(0.6)

    # Battle result
    if player.is_alive():
        print(f"Victory! {enemy.name} defeated.")
        xp_gain = enemy.level * 25
        player.xp += xp_gain
        print(f"You gain {xp_gain} XP.")
        # level up if xp threshold reached
        leveled = check_level_up(player)
        return True
    else:
        print("You were defeated...")
        return False

def print_hud(player: Character, enemy: Character, inventory: Dict[str, Item]):
    def hpbar(cur, mx, width=20):
        filled = int(cur/mx * width)
        return '[' + '#' * filled + '-' * (width - filled) + ']'
    print(f"{player.name} (Lv {player.level})  HP: {player.hp}/{player.max_hp} {hpbar(player.hp, player.max_hp)}")
    print(f"{enemy.name} (Lv {enemy.level})   HP: {enemy.hp}/{enemy.max_hp} {hpbar(enemy.hp, enemy.max_hp)}\n")
    print("Your statuses:", ', '.join([f"{s.name}({s.duration})" for s in player.statuses]) or "None")
    print("Enemy statuses:", ', '.join([f"{s.name}({s.duration})" for s in enemy.statuses]) or "None")
    print("\nInventory:")
    for k, it in inventory.items():
        print(f" - {it.name} ({k}): {it.qty} : {it.desc}")
    print()

def player_action(player: Character, enemy: Character, inventory: Dict[str, Item]):
    # Show player's moves and options
    while True:
        print("Choose an action:")
        print("1) Use Move")
        print("2) Use Item")
        print("3) Pass")
        print("4) Stats")
        choice = input("> ").strip()
        if choice == '1':
            # Moves list
            for i, m in enumerate(player.moves, 1):
                cd_info = f"(CD:{m.current_cd})" if m.current_cd > 0 else ""
                print(f"{i}) {m.name} {cd_info} - {m.desc}")
            sel = input("Select move number: ").strip()
            if not sel.isdigit() or not (1 <= int(sel) <= len(player.moves)):
                print("Invalid selection.")
                continue
            move = player.moves[int(sel)-1]
            if not move.ready():
                print(f"{move.name} is on cooldown ({move.current_cd} turns left).")
                continue
            perform_move(player, enemy, move, target=enemy if not move.target_self else player)
            return
        elif choice == '2':
            # Items
            available = [k for k, v in inventory.items() if v.qty > 0]
            if not available:
                print("No items available.")
                continue
            for idx, k in enumerate(available, 1):
                it = inventory[k]
                print(f"{idx}) {it.name} x{it.qty} - {it.desc}")
            sel = input("Select item number: ").strip()
            if not sel.isdigit() or not (1 <= int(sel) <= len(available)):
                print("Invalid selection.")
                continue
            key = available[int(sel)-1]
            item = inventory[key]
            print(item.use(player))
            item.qty -= 1
            return
        elif choice == '3':
            print(f"{player.name} chooses to pass this turn.")
            return
        elif choice == '4':
            print_character_stats(player)
            input("Press Enter to continue...")
            continue
        else:
            print("Invalid choice, try again.")

def enemy_action(enemy: Character, player: Character):
    move = enemy_choose_move(enemy, player)
    if not move:
        print(f"{enemy.name} seems unable to act.")
        return
    # If move has cooldown, set it
    perform_move(enemy, player, move, target=player)

def perform_move(user: Character, target: Character, move: Move, target_is_enemy=True, target_character: Optional[Character]=None, target=None):
    # here 'target' is explicit param
    if target is None:
        # default target param earlier was used
        pass
    # Resolve hit
    print(f"{user.name} uses {move.name}!")
    if not attempt_hit(move):
        print("But it missed!")
        move.trigger_cd()
        time.sleep(0.4)
        return

    if move.power < 0:  # healing move
        amt = -move.power
        healed = user.heal(amt)
        print(f"{user.name} heals {healed} HP.")
    else:
        dmg = calculate_damage(user, target, move.power)
        inflicted = target.take_damage(dmg)
        print(f"It deals {inflicted} damage to {target.name}! ({target.hp}/{target.max_hp})")

    # apply status from move if any
    if move.apply_status:
        s = move.apply_status(user, target)
        if s:
            target.add_status(s)
            print(f"{target.name} is afflicted with {s.name} for {s.duration} turns!")

    # apply buff/debuff
    if move.buff_stat:
        if move.target_self:
            user.add_buff(move.buff_stat, move.buff_amount, move.buff_duration)
            print(f"{user.name}'s {move.buff_stat} increased by {move.buff_amount} for {move.buff_duration} turns.")
        else:
            target.add_buff(move.buff_stat, -move.buff_amount, move.buff_duration)
            print(f"{target.name}'s {move.buff_stat} decreased by {move.buff_amount} for {move.buff_duration} turns.")

    # set cooldown
    move.trigger_cd()
    time.sleep(0.6)

def print_character_stats(ch: Character):
    print(f"--- {ch.name} Stats ---")
    print(f"Level: {ch.level}")
    print(f"HP: {ch.hp}/{ch.max_hp}")
    print(f"ATK: {ch.get_stat('atk')}")
    print(f"DEF: {ch.get_stat('defense')}")
    print(f"SPD: {ch.get_stat('speed')}")
    print("Moves:")
    for m in ch.moves:
        print(f" - {m.name}: {m.desc} (Power: {m.power}, Acc: {m.accuracy}, CD: {m.cooldown})")
    print("----------------------")

# ---- Leveling ----
def check_level_up(player: Character):
    leveled = False
    # simple rule: every 100 xp => level up
    while player.xp >= 100:
        player.xp -= 100
        player.level += 1
        leveled = True
        # stat increases
        hp_gain = random.randint(6, 12)
        atk_gain = random.randint(1, 3)
        def_gain = random.randint(1, 3)
        spd_gain = random.randint(0, 2)
        player.max_hp += hp_gain
        player.atk += atk_gain
        player.defense += def_gain
        player.speed += spd_gain
        player.hp = player.max_hp  # heal to full on level
        print(f"\n*** {player.name} leveled up to Lv {player.level}! ***")
        print(f"+{hp_gain} HP, +{atk_gain} ATK, +{def_gain} DEF, +{spd_gain} SPD\n")
    return leveled

# ---- Game Loop / Encounters ----
def generate_enemy(player_level: int):
    # scale enemy with a bit of randomness
    lvl = max(1, player_level + random.choice([-1, 0, 1]))
    base_hp = 50 + lvl * 8
    enemy = Character(
        name=random.choice(['Goblin', 'Wolf', 'Bandit', 'Slime', 'Rogue']),
        level=lvl,
        max_hp=base_hp,
        hp=base_hp,
        atk=8 + lvl * 2,
        defense=4 + lvl,
        speed=5 + lvl,
        moves=make_enemy_moves(lvl)
    )
    return enemy

def main():
    clear_screen()
    print("Welcome to TurnQuest — a demo turn-based battle system!")
    player_name = input("Choose your hero's name: ").strip() or "Hero"
    # starter player
    player = Character(
        name=player_name,
        level=1,
        max_hp=100,
        hp=100,
        atk=12,
        defense=8,
        speed=10,
        moves=make_player_moves()
    )
    inventory = make_items()

    # Simple adventure loop: face 3 enemies to win, or loop until player dies
    encounters = 0
    while True:
        encounters += 1
        enemy = generate_enemy(player.level)
        result = battle(player, enemy, inventory)
        if not result:
            print("Game over. Try again!")
            break
        # after victory, small reward: maybe item drop
        if random.random() < 0.4:
            inventory['potion'].qty += 1
            print("You found a Potion!")
        if random.random() < 0.15:
            inventory['antidote'].qty += 1
            print("You found an Antidote!")
        print()
        # after a few encounters, raise challenge or end
        if encounters >= 5:
            print("You've cleared 5 encounters. Do you want to (C)ontinue adventuring or (Q)uit as a hero?")
            c = input("> ").strip().lower()
            if c.startswith('q'):
                print("You end your adventure victorious. Goodbye!")
                break
            else:
                print("You press on into more danger...")
        # small rest option between fights
        print("Do you want to (R)est to recover 30 HP, (S)tatus, or (C)ontinue?")
        choice = input("> ").strip().lower()
        if choice.startswith('r'):
            healed = player.heal(30)
            print(f"You rest and recover {healed} HP. ({player.hp}/{player.max_hp})")
        elif choice.startswith('s'):
            print_character_stats(player)
            input("Press Enter to continue...")
        # continue the adventure

if __name__ == "__main__":
    main()
